In [1]:
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile


In [2]:
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
from data_utils import get_CIFAR10_data
import resource
import numpy as np
from numpy import float32
import gzip

# reload(data_utils)
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [3]:
"""
Helper methods from tensorflow examples to read MNIST images
"""

def _read32(bytestream):
  dt = np.dtype(np.uint32).newbyteorder('>')
  return np.frombuffer(bytestream.read(4), dtype=dt)[0]

def extract_images(filename):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
  print('Extracting', filename)
  with gfile.Open(filename, 'rb') as f, gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2051:
      raise ValueError('Invalid magic number %d in MNIST image file: %s' %
                       (magic, filename))
    num_images = _read32(bytestream)
    rows = _read32(bytestream)
    cols = _read32(bytestream)
    buf = bytestream.read(rows * cols * num_images)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(num_images, rows, cols, 1)
    return data

def extract_labels(filename, one_hot=False, num_classes=10):
  """Extract the labels into a 1D uint8 numpy array [index]."""
  print('Extracting', filename)
  with gfile.Open(filename, 'rb') as f, gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError('Invalid magic number %d in MNIST label file: %s' %
                       (magic, filename))
    num_items = _read32(bytestream)
    buf = bytestream.read(num_items)
    labels = np.frombuffer(buf, dtype=np.uint8)
    if one_hot:
      return dense_to_one_hot(labels, num_classes)
    return labels

def dense_to_one_hot(labels_dense, num_classes):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot



In [57]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv2d_relu(x, W, b):
    return tf.nn.relu(conv2d(x,W) + b)

def conv2d_batch_relu(x, W, b, n_out, phase_train):
    conv = conv2d(x,W) + b
    conv_bn = batch_norm(conv, n_out, phase_train)
    return tf.nn.relu(conv_bn)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def weight_variable(shape, name, stddev=1e-3):    
    initial = tf.truncated_normal(shape, stddev=stddev, name=name)

    return tf.Variable(initial)

def bias_variable(shape, name):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial, name=name)


In [5]:
def batch_norm(x, n_out, phase_train, scope='bn', dims=[0,1,2]):
    """
    Batch normalization on convolutional maps.
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope(scope):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, dims, name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [ ]:
#todo: calculate size reduced after maxpool
#todo: change accuracy check on training to rando 1000
# Check weight initialization
# Add batch norm to FC

# num_epochs = 200
# num_train = 100
# batch_size = 50
# num_classes = 10

num_epochs = 150
num_train = 49000
batch_size = 200
num_classes = 10

small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}
X = small_data['X_train'].astype(float32)
X = np.reshape(X, (X.shape[0], -1))
y = small_data['y_train']

X_val = small_data['X_val'].astype(float32)
X_val = np.reshape(X_val, (X_val.shape[0], -1))
y_val = small_data['y_val']


W,H = 32,32
C = 3
input_dim = C * W * H
num_conv1_filters = 32
num_conv2_filters = 64


f_size = 5
fc_hidden_dim = 500

"""
Placeholders
"""
x_ = tf.placeholder(tf.float32, shape=[None, input_dim], name="x_placeholder")
x = tf.reshape(x_, [-1,H,W,C]) # double check if this reshape hasn't messed anything up:
y_ = tf.placeholder(tf.int64, shape=(None), name="y_placeholder")
keep_prob = tf.placeholder(tf.float32, name="keep_prob_placeholder")
phase_train = tf.placeholder(tf.bool, name='phase_train')

"""
Conv layer 1
"""
W_conv1_1 = weight_variable((5, 5, C, num_conv1_filters), name="W_conv1_1")
# XXX: left off here, figure out how to best handle auto generation of W.  nu_conv1_filters needs to match with next conv3
W_conv1_2 = weight_variable((5, 5, num_conv1_filters, num_conv1_filters), name="W_conv1_2")
b_1_1 = bias_variable([num_conv1_filters], name='b1_1')
b_1_2 = bias_variable([num_conv1_filters], name='b1_2')

print "conv1"
# old one before batch norm: h_conv_relu_1 = conv2d_relu(x, W_conv1_1, b_1_1)

h_conv_relu_1 = conv2d_batch_relu(x, W_conv1_1, b_1_1, num_conv1_filters, phase_train)
# old: h_conv_relu_2 = conv2d_relu(h_conv_relu_1, W_conv1_2, b_1_2)
h_conv_relu_2 = conv2d_batch_relu(h_conv_relu_1, W_conv1_2, b_1_2, num_conv1_filters, phase_train)
h_pool = max_pool_2x2(h_conv_relu_2)
print "--"
# print h_pool


"""
Conv layer 2
"""
W_conv2 = weight_variable( (5, 5, num_conv1_filters, num_conv2_filters), name="W_conv2" )
b_2 = bias_variable([num_conv2_filters], name='b2')

print "conv 2"
h_conv_relu_2_1 = conv2d_batch_relu(h_pool, W_conv2, b_2, num_conv2_filters, phase_train)
h_conv_pool_2_1 = max_pool_2x2(h_conv_relu_2_1)


"""
Conv layer 3
"""
W_conv3 = weight_variable((5, 5, num_conv2_filters, num_conv2_filters), name="W_conv3")
b_3 = bias_variable([num_conv2_filters], name='b3')

h_conv_relu_3 = conv2d_batch_relu(h_conv_pool_2_1, W_conv3, b_3, num_conv2_filters, phase_train)
h_conv_pool_3 = max_pool_2x2(h_conv_relu_3)
print "*"*10
print h_conv_pool_3
print "*"*10
"""
FC Layer
"""
# double check the 16 number
# W_fc1 = tf.Variable(1e-3*tf.truncated_normal((16*16*num_conv1_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")

#XXX:  8*8, now 4*4 needs to match output of max_pool
# W_fc1 = tf.Variable(1e-2*tf.truncated_normal((8*8*num_conv2_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
W_fc1 = tf.Variable(1e-2*tf.truncated_normal((4*4*num_conv2_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.zeros([fc_hidden_dim]), name="b_fc1")
# h_pool_flat = tf.reshape(h_pool, [-1, 16*16*32])
# h_pool_flat = tf.reshape(h_conv_pool_3, [-1, 8*8*num_conv2_filters])

# XXX: when automating, this 4x4 needs to be calculated automatically because max pool resizes it
h_pool_flat = tf.reshape(h_conv_pool_3, [-1, 4*4*num_conv2_filters])
print h_pool_flat
print W_fc1
# (10,8192) * (8192,500) = (10,500)

h_fc1_forward = tf.matmul(h_pool_flat, W_fc1) + b_fc1
h_bn_fc1 = batch_norm(h_fc1_forward, 500, phase_train, dims=[0])
h_fc1 = tf.nn.relu(h_bn_fc1)
print h_fc1

"""
Dropout
"""
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

"""
Output Layer
"""
W_fc2 = tf.Variable(1e-2*tf.truncated_normal((fc_hidden_dim,num_classes), stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.zeros([num_classes]), name="b_fc2")
scores_fc2 = (tf.matmul(h_fc1, W_fc2) + b_fc2)
print scores_fc2
# y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)
# print y_conv
# print tf.log(y_conv)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(scores_fc2, y_, name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
print cross_entropy
print loss


# optimizer = tf.train.GradientDescentOptimizer(1e-2)
# # Create a variable to track the global step.
# global_step = tf.Variable(0, name='global_step', trainable=False)
# train_op = optimizer.minimize(loss, global_step=global_step)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

with tf.Session() as session:
    session.run(tf.initialize_all_variables())

    for epoch in xrange(num_epochs):
        for i in xrange(num_train/batch_size):
            batch_xs = X[i*batch_size:i*batch_size+batch_size] # (batchsize, image_dim)
            batch_ys = y[i*batch_size:i*batch_size+batch_size] # (batchsize,)

            feed_train = {x_: batch_xs, y_: batch_ys, keep_prob:0.5, phase_train.name: True}
            session.run(train_op, feed_dict=feed_train)
            
        print session.run(loss, feed_dict=feed_train)
    
        # sample at random for testing training
        batch_mask = np.random.choice(X.shape[0], batch_size)
        feed_train = {x_: X[batch_mask], y_: y[batch_mask], keep_prob:1, phase_train.name: False}
        feed_val = {x_: X_val, y_: y_val, keep_prob:1, phase_train.name: False}
        
        correct_prediction = tf.equal(tf.argmax(scores_fc2, 1), y_)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_acc_result = accuracy.eval(feed_train)
        vac_acc_results = accuracy.eval(feed_val)
        print("(Epoch: {e}/{et}) train acc: {tacc}; val_acc: {vacc}".format(
                e=epoch+1,et=num_epochs, tacc=train_acc_result, vacc=vac_acc_results))

    

In [ ]:
X.shape
batch_mask = (np.random.choice(X.shape[0], 200))
X[mask].shape


In [41]:
def run_conv(X, y, X_val, y_val, num_conv_layers, num_conv_filters, fc_hidden_dim, H, W, C, epochs, learning_rate, lr_decay):
    print "batchsize:"
    print batch_size
    num_classes = 10
    input_dim = C * W * H
#     num_conv1_filters = num_conv_filters
    output_size=W

    # Filter size:
    f_size = 3 
    
    # Number of neurons in fully-connected layer
    fc_hidden_dim = fc_hidden_dim

    # Placeholders:
    x_ = tf.placeholder(tf.float32, shape=[None, input_dim], name="x_placeholder")
    y_ = tf.placeholder(tf.int64, shape=(None), name="y_placeholder")
#     keep_prob = tf.placeholder(tf.float32, name="keep_prob_placeholder")
    keep_prob = tf.placeholder(tf.float32)
    phase_train = tf.placeholder(tf.bool, name='phase_train')

    x = x_
    x = tf.reshape(x_, [-1,H,W,C]) # double check if this reshape hasn't messed anything up
    conv_out = None
    
    """
    Begin constructing arbitrary number of conv-batchnorm-relu-pool layers.
    We are going to create sets of two.  Meaning, if num_conv_layers = 2
    then this will create 6 layers:
    conv-batchnorm-relu-pool-conv-batchnorm-relu-pool
    """
    for i in xrange(num_conv_layers):
        scope_name = 'convolutions_{index}'.format(index=i)
        print scope_name
        with tf.name_scope(scope_name):
            if i == 0:
                filters = C
                num_conv1_filters = num_conv_filters[i]
            else:
                filters = num_conv_filters[i-1]
                num_conv1_filters = num_conv_filters[i]
#                 filters = num_conv_filters

            w = weight_variable( (f_size, f_size, filters, num_conv1_filters), name="w")
            w2 = weight_variable( (f_size, f_size, num_conv1_filters, num_conv1_filters), name="w2")
            b = bias_variable([num_conv1_filters], name='b')
            b2 = bias_variable([num_conv1_filters], name='b2')

            # Conv-Relu-Conv-Relu-Pool
            conv1 = conv2d_batch_relu(x, w, b, num_conv1_filters, phase_train)
            conv1_do = tf.nn.dropout(conv1, keep_prob)
            
            conv2 = conv2d_batch_relu(conv1_do, w2, b2, num_conv1_filters, phase_train)
            pool = max_pool_2x2(conv2)
            x = pool
            print x
            print "{f1} -> {f2}".format(f1=filters,f2=num_conv1_filters)
            output_size/=2 
            print output_size
            conv_out = x

    """
    FC Layer
    """
    #XXX:  8*8, now 4*4 needs to match output of max_pool
    # W_fc1 = tf.Variable(1e-2*tf.truncated_normal((8*8*num_conv2_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
    W_fc1 = tf.Variable(1e-3*tf.truncated_normal((output_size*output_size*num_conv1_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
    b_fc1 = tf.Variable(tf.zeros([fc_hidden_dim]), name="b_fc1")

    # h_pool_flat = tf.reshape(h_pool, [-1, 16*16*32])
    # h_pool_flat = tf.reshape(h_conv_pool_3, [-1, 8*8*num_conv2_filters])
    # XXX: when automating, this 4x4 needs to be calculated automatically because max pool resizes it

    h_pool_flat = tf.reshape(conv_out, [-1, output_size*output_size*num_conv1_filters])
    h_fc1_forward = tf.matmul(h_pool_flat, W_fc1) + b_fc1
    h_bn_fc1 = batch_norm(h_fc1_forward, fc_hidden_dim, phase_train, dims=[0])
    h_fc1 = tf.nn.relu(h_bn_fc1)

    """
    Dropout
    """
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    """
    Output Layer
    """
    W_fc2 = weight_variable((fc_hidden_dim,num_classes), name="W_fc2", stddev=1e-3)
    
    b_fc2 = tf.Variable(tf.zeros([num_classes]), name="b_fc2")
    scores_fc2 = (tf.matmul(h_fc1, W_fc2) + b_fc2)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(scores_fc2, y_, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())
        for epoch in xrange(num_epochs):
            for i in xrange(num_train/batch_size):
                batch_xs = X[i*batch_size:i*batch_size+batch_size] # (batchsize, image_dim)
                batch_ys = y[i*batch_size:i*batch_size+batch_size] # (batchsize,)

                feed_train = {x_: batch_xs, y_: batch_ys, 
                              keep_prob:0.5, phase_train.name: True}
                if i==0:
                    print "loss:"
                    print session.run(loss, feed_dict=feed_train)
                session.run(train_op, feed_dict=feed_train)

            # sample at random for testing training
            batch_mask = np.random.choice(X.shape[0], batch_size)
            feed_train = {x_: X[batch_mask], y_: y[batch_mask], keep_prob:1, phase_train.name: False}
            feed_val = {x_: X_val, y_: y_val, keep_prob:1, phase_train.name: False}

            correct_prediction = tf.equal(tf.argmax(scores_fc2, 1), y_)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            train_acc_result = accuracy.eval(feed_train)
            vac_acc_results = accuracy.eval(feed_val)
            print("(Epoch: {e}/{et}) train acc: {tacc}; val_acc: {vacc}".format(
                    e=epoch+1,et=num_epochs, tacc=train_acc_result, vacc=vac_acc_results))
    #         val_acc_result = accuracy.eval(feed_val)

In [34]:
"""
Training MNIST data
"""

num_epochs = 10
num_train = 55000
batch_size = 5
VALIDATION_SIZE=5000

# This will create 6 layers (conv-batchnorm-relu-pool-conv-batchnorm-relu-pool)
num_conv_layers = 2   
num_conv_filters = [32]
fc_hidden_dim = 200

train_images = extract_images("./MNIST_data/train-images-idx3-ubyte.gz")
train_labels = extract_labels("./MNIST_data/train-labels-idx1-ubyte.gz", one_hot=False)
validation_images = train_images[:VALIDATION_SIZE]
validation_labels = train_labels[:VALIDATION_SIZE]
train_images = train_images[VALIDATION_SIZE:]
train_labels = train_labels[VALIDATION_SIZE:]
print train_labels.shape


X = train_images
X = np.reshape(X, (X.shape[0], -1))
X = X[:num_train]
y = train_labels
X_val = validation_images
X_val = np.reshape(X_val, (X_val.shape[0], -1))
y_val = validation_labels
W,H,C = 28,28,1

run_conv(X, y, X_val, y_val, num_conv_layers, num_conv_filters, H, W, C, num_epochs)

('Extracting', './MNIST_data/train-images-idx3-ubyte.gz')
('Extracting', './MNIST_data/train-labels-idx1-ubyte.gz')
(55000,)


TypeError: run_conv() takes exactly 13 arguments (10 given)

In [63]:
"""
CIFAR-10 data
"""
# 49000
num_epochs = 200
num_train = 49000
batch_size = 200
num_conv_layers = 2   
# num_conv_filters = [32, 64, 128]
num_conv_filters = [32, 64]
fc_hidden_dim = 1024
learning_rate = 1e-3
lr_decay = .95
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}
X = small_data['X_train'].astype(float32)

add_images = np.zeros(X.shape)
for idx in xrange(X.shape[0]):
    add_images[idx] = np.fliplr(X[idx])

num_train*=2
print "now:"
print num_train

X = np.concatenate([X,add_images])
X = np.reshape(X, (X.shape[0], -1))
y = small_data['y_train']

copied = np.array(y, copy=True)
y = np.concatenate([y,copied])

X_val = small_data['X_val'].astype(float32)
X_val = np.reshape(X_val, (X_val.shape[0], -1))
y_val = small_data['y_val']

W,H,C = 32,32,3
print X.shape
run_conv(X, y, X_val, y_val, num_conv_layers, num_conv_filters, fc_hidden_dim, H, W, C, 
         num_epochs, learning_rate, lr_decay)

now:
98000
(98000, 3072)
batchsize:
200
convolutions_0
Tensor("convolutions_0_28/MaxPool:0", shape=(?, 16, 16, 32), dtype=float32)
3 -> 32
16
convolutions_1
Tensor("convolutions_1_2/MaxPool:0", shape=(?, 8, 8, 64), dtype=float32)
32 -> 64
8
loss:
2.30258
(Epoch: 1/200) train acc: 0.465000003576; val_acc: 0.321000009775
loss:
2.06716
(Epoch: 2/200) train acc: 0.569999992847; val_acc: 0.354999989271
loss:
1.94416
(Epoch: 3/200) train acc: 0.569999992847; val_acc: 0.386999994516
loss:
1.64959
(Epoch: 4/200) train acc: 0.615000009537; val_acc: 0.374000012875
loss:
1.59603
(Epoch: 5/200) train acc: 0.685000002384; val_acc: 0.389999985695
loss:
1.43884
(Epoch: 6/200) train acc: 0.625; val_acc: 0.395999997854
loss:
1.3002
(Epoch: 7/200) train acc: 0.694999992847; val_acc: 0.430000007153
loss:
1.07222
(Epoch: 8/200) train acc: 0.730000019073; val_acc: 0.432999998331
loss:
0.881613
(Epoch: 9/200) train acc: 0.714999973774; val_acc: 0.430999994278
loss:
0.791456
(Epoch: 10/200) train acc: 0.7799

KeyboardInterrupt: 